<a href="https://colab.research.google.com/github/JeongSoong/Machine_learning/blob/main/%EB%AC%B8%EC%A0%95%EC%88%98_201911047_8%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train,y_train),(x_valid,y_valid) = mnist.load_data()

def plot_image(data, idx):
    plt.figure(figsize = (5,5))
    plt.imshow(data[idx], cmap = 'gray')
    plt.axis('off')
    plt.show()
plot_image(x_train, 0)

In [ ]:
print(x_train.min(), x_train.max())
print(x_valid.min(), x_valid.max())

In [ ]:
##정규화 실시
x_train = x_train/x_train.max()
x_valid = x_valid/x_valid.max()

In [ ]:
print(x_train.min(), x_train.max())
print(x_valid.min(), x_valid.max())

In [ ]:
x_train.shape

In [ ]:
##마지막에 흑백 생상 채널 추가
x_train_in = x_train[..., tf.newaxis]
x_valid_in = x_valid[..., tf.newaxis]
print(x_train_in.shape, x_valid_in.shape)

In [ ]:
##Conv20, kernal : 3X3, name: 레이어 이름 conv, 활성화함수: ReLu
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32,(3,3), activation = 'relu', input_shape = (28,28,1),name = 'conv'),
    tf.keras.layers.MaxPool2D((2,2), name = 'pool'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10,activation = 'softmax')
])

In [ ]:
model.compile(optimizer='adam',
                    loss = 'sparse_categorical_crossentropy',
                    metrics=['accuracy'])
history = model.fit(x_train,y_train, validation_data = (x_valid, y_valid), epochs = 10)

In [ ]:
model.summary()

In [ ]:
model.evaluate(x_valid_in, y_valid)

In [ ]:
loss, val_loss = history.history['loss'], history.history['val_loss']
acc, val_acc = history.history['accuracy'], history.history['val_accuracy']
epoch = 10
fig, axes = plt.subplots(1,2,figsize = (12,4))
axes[0].plot(range(1, epoch+1), loss, label = 'Training')
axes[0].plot(range(1, epoch+1), val_loss, label = 'Validation')
axes[0].legend(loc = 'best')
axes[0].set_title('Loss')

axes[1].plot(range(1, epoch+1), acc, label = 'Training')
axes[1].plot(range(1, epoch+1), val_acc, label = 'Validation')
axes[1].legend(loc = 'best')
axes[1].set_title('Accuracy')

plt.show()

모델 구조 파악


1.   입력형태(28,28,1)
2.   출력형태(10)
3.   첫번쨰 레이어: 합성곱 Conv2D, 가중치(weight)(3,3,1,32)3332=280개, 상수항(bias)32/output(26,26,1,32)
4.   두번째 레이어: Pooling(26,26,1,32)
5.   세번째 레이어: 1차 배열로 변화
6.   네번째 레이어: 출력



In [ ]:
##각 층에 대한 정보
model.layers

In [ ]:
##첫번째 레이어(Conv2D) 가중치: 3*3*32 = 280개
model.layers[0].weights

In [ ]:
##첫번째 레이어(Conv2D) 상수항: 32개
model.layers[0].bias

In [ ]:
##첫번쨰 레이어의 커널 가중치
model.layers[0].kernel

In [ ]:
model.layers[0].bias

In [ ]:
##Conv2D 레이어 이름으로 소환하기
model.get_layer('conv')

In [ ]:
##Conv2D와 Pooling 레이어의 output(이미지) 출력
##모델 만들기: tf.keras.Model(입력 = 입력 데이터, 출력 = 첫번째, 두번째 층의 출력력)
##모델 예측: 모델명.predict()
activator = tf.keras.Model(inputs=model.input,outputs = [layer.output for layer in model.layers[:2]])

In [ ]:
##입력데이터 첫번째 5에 대해 출력
activations = activator.predict(x_train_in[0][tf.newaxis, ...])
len(activations)
##Conv2D층과 Pooling층의 output이므로 '2'라는 결과

In [ ]:
##activations[0]
conv = activations[0]
conv.shape

In [ ]:
##'5'의 첫번째 Conv2D를 통과한 32개 kernel별 특성맵 시각화
## Convolution 시각화
fig, axes = plt.subplots(4,8)
fig.set_size_inches(10, 5)

for i in range(32):
    ax = axes[i//8, i%8]
    ax.imshow(conv[0, :, :, i], cmap = 'viridis')
    ax.set_title('kernel %s'%str(i), fontsize=10)
    plt.setp( axes[i//8, i%8].get_xticklabels(), visible=False)
    plt.setp( axes[i//8, i%8].get_yticklabels(), visible=False)

plt.tight_layout()
plt.show()

In [ ]:
#activations[1]
pooling = activations[1]
pooling.shape

In [ ]:
##'5'의 두번째 Pooling을 통과한 32개 kernel별 특성맵 시각화
#pooling 시각화
fig, axes = plt.subplots(4,8)
fig.set_size_inches(10, 5)

for i in range(32):
    ax = axes[i//8, i%8]
    ax.imshow(conv[0, :, :, i], cmap = 'viridis')
    ax.set_title('kernel %s'%str(i), fontsize=10)
    plt.setp( axes[i//8, i%8].get_xticklabels(), visible=False)
    plt.setp( axes[i//8, i%8].get_yticklabels(), visible=False)

plt.tight_layout()
plt.show()

In [ ]:
import tensorflow as tf
import tensorflow_hub as tfhub

In [ ]:
img_path = 'https://upload.wikimedia.org/wikipedia/commons/thumb/c/c4/Gangnam_Seoul_January_2009.jpg/1280px-Gangnam_Seoul_January_2009.jpg'
img = tf.keras.utils.get_file(fname='gangnam', origin = img_path)
img = tf.io.read_file(img)  #파일 객체를 string으로 변환
img = tf.image.decode_jpeg(img, channels=3)  #문자(string)를 숫자(unit8)텐서로 변환
img = tf.image.convert_image_dtype(img, tf.float32)  #0~1 범위로 정규화

import matplotlib.pylab as plt
plt.figure(figsize=(15, 10))
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
img_input = tf.expand_dims(img, 0) #batch_size추가, 4차원 텐서로 입력
img_input.shape

In [ ]:
##TensorFlow Hub에서 모델 가져오기 - FasterRCNN+InceptionResNet V2
model = tfhub.load("https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1")

In [ ]:
#모델 시그니처(용도) 확인
model.signatures.keys()

In [ ]:
#객체탐지 모델 생성
obj_detector = model.signatures['default']
obj_detector

In [ ]:
#모델을 이용하여 예측(추론)
result = obj_detector(img_input)
result.keys()

In [ ]:
type(result)

In [ ]:
labels = result["detection_class_labels"]
names = result['detection_class_names']

In [ ]:
boxes = result["detection_boxes"]  #Bounding Box 좌표 예측값
labels = result["detection_class_entities"]  #클래스 값
scores = result["detection_scores"]  #신뢰도 (confidence)

In [ ]:
#탐지한 객체의 개수
len(result["detection_scores"])

In [ ]:
result.keys()

In [ ]:
for key, value in result.items():
    print(key, value)

In [ ]:
img_height, img_width = img.shape[0], img.shape[1]
obj_to_detect = 10
for i in range(min(obj_to_detect, boxes.shape[0])):
    if scores[i] >= 0.2:
        (ymax, xmin, ymin, xmax) = (boxes[i][0]*img_height, boxes[i][1]*img_width,
                                    boxes[i][2]*img_height, boxes[i][3]*img_width)
        print("{} : ({}, {}, {}, {})". format(scores[i], ymax, xmin, ymin, xmax))

In [ ]:
#객체 탐지 결과 시각화
boxes = result["detection_boxes"]  #Bounding Box 좌표 예측값
labels = result["detection_class_entities"]  #클래스 값
scores = result["detection_scores"]  #신뢰도 (confidence)

#샘플 이미지 가로 세로 크기
img_height, img_width = img.shape[0], img.shape[1]

#탐지할 최대 객체의 수
obj_to_detect = 10

#시각화
plt.figure(figsize = (15, 10))
for i in range(min(obj_to_detect, boxes.shape[0])):
    if scores[i] >= 0.2:
        (ymax, xmin, ymin, xmax) = (boxes[i][0]*img_height, boxes[i][1]*img_width,
                                    boxes[i][2]*img_height, boxes[i][3]*img_width)
        
        plt.imshow(img)
        plt.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin],
                 color = 'yellow', linewidth=2)
        
        class_name = labels[i].numpy().decode('utf-8')
        infer_score = str(scores[i].numpy()*100)
        annotation = "{}: {}%", format(class_name, infer_score)
        plt.text(xmin+10, ymax+20, annotation,
                 color='white', backgroundcolor='blue', fontsize=10)